# AlexNet

Alexnet is a convolution neural network the competed in ImageNet Large Scale Visual Recognition Challenge in 2012. It uses features like convolution, max pooling, LRN, dropout etc. For more information [ImageNet Classification with Deep Convolutional Neural Networks](https://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf) by Alex Krizhevsky, Ilya Sutskever and Geoffrey E. Hinton

## Architecture
input -> conv11-96 -> maxpooling -> LRN -> conv5-256 -> maxpooling -> LRN -> conv3-384 -> conv3-384 -> conv3-256 -> maxpooling -> LRN -> flatten -> fc4096 -> dropout -> fc4096 -> dropout -> output

### Note :
Because of the small image sizes, paddings are changed to "SAME".

In [1]:
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers import Input
from keras.layers import Activation
from keras.models import Model
from keras.utils import to_categorical
from keras.optimizers import SGD
from keras.callbacks import TensorBoard

Using TensorFlow backend.


In [2]:
from data import get_data_set

In [3]:
# Args
save_dir = "./results"
batch_size = 64
epochs = 100
lr=0.001
decay=0.0005
momentum=0.9
log_dir = "./tensorboard/alexnet_9x9_12/"
# originals
# save_dir = "./results"
# batch_size = 50
# epochs = 50
# lr=0.01
# decay=0.0005
# momentum=0.9

In [4]:
X_train,Y_train = get_data_set("train",input_path = "../input/data_9x9_12band/",one_hot = True)
X_test,Y_test = get_data_set("test",input_path = "../input/data_9x9_12band/", one_hot = True)

X_train.shape

(404, 9, 9, 12)

In [5]:
tensorboard = TensorBoard(log_dir=log_dir,batch_size=batch_size)

# Custome LRN or Batch normalization

There are no local response normalization layer in Keras' latest version when this code is written. Instead of LRN, batch normalization is used. 
A custom LRN can be implemented if needed. An implementation example can be found in the [link](https://joelouismarino.github.io/blog_posts/blog_googlenet_keras.html)

# Model

In [6]:
inputs = Input(X_train.shape[1:])
inputs

<tf.Tensor 'input_1:0' shape=(?, 9, 9, 12) dtype=float32>

In [7]:
conv_1 = Convolution2D(96, kernel_size =(11, 11), strides=(4, 4), padding='same', activation='relu',
                           name='conv_1')(inputs)
conv_1 = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(conv_1)
norm_1 = BatchNormalization(name="norm_1")(conv_1)
norm_1

<tf.Tensor 'norm_1/cond/Merge:0' shape=(?, 2, 2, 96) dtype=float32>

In [8]:
conv_2 = Convolution2D(256, kernel_size =(5, 5), strides=(1, 1), padding='same', activation='relu',
                           name='conv_2')(norm_1)
conv_2 = MaxPooling2D((3, 3), strides=(2, 2),padding = "same")(conv_2)
norm_2 = BatchNormalization(name="norm_2")(conv_2)
norm_2

<tf.Tensor 'norm_2/cond/Merge:0' shape=(?, 1, 1, 256) dtype=float32>

In [9]:
conv_3 = Convolution2D(384, kernel_size =(3, 3), strides=(1, 1), padding='same', activation='relu',
                           name='conv_3')(conv_2)
conv_4 = Convolution2D(384, kernel_size =(3, 3), strides=(1, 1), padding='same', activation='relu',
                           name='conv_4')(conv_3)
conv_5 = Convolution2D(256, kernel_size =(3, 3), strides=(1, 1), padding='same', activation='relu',
                           name='conv_5')(conv_4)
conv_5 = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(conv_5)
norm_3 = BatchNormalization(name="norm_3")(conv_5)
norm_3

<tf.Tensor 'norm_3/cond/Merge:0' shape=(?, 1, 1, 256) dtype=float32>

Flatten

In [10]:
dense_1 = Flatten(name='flatten')(norm_3)
dense_1

<tf.Tensor 'flatten/Reshape:0' shape=(?, ?) dtype=float32>

In [11]:
dense_1 = Dense(4096, activation='relu', name='dense_1')(dense_1)
dense_2 = Dropout(0.5)(dense_1)
dense_2 = Dense(4096, activation='relu', name='dense_2')(dense_2)
dense_3 = Dropout(0.5)(dense_2)
dense_3 = Dense(Y_train.shape[1], name='dense_3')(dense_3)
prediction = Activation('softmax', name='softmax')(dense_3)
prediction

<tf.Tensor 'softmax/Softmax:0' shape=(?, 4) dtype=float32>

In [12]:
model = Model(outputs=prediction,inputs = inputs)

# Lets train

In [13]:
# optimizer
sgd = SGD(lr=lr, decay=decay, momentum=momentum)

In [14]:
model.compile(optimizer=sgd, loss='categorical_crossentropy',metrics=['accuracy'])

In [16]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 9, 9, 12)          0         
_________________________________________________________________
conv_1 (Conv2D)              (None, 3, 3, 96)          139488    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 2, 2, 96)          0         
_________________________________________________________________
norm_1 (BatchNormalization)  (None, 2, 2, 96)          384       
_________________________________________________________________
conv_2 (Conv2D)              (None, 2, 2, 256)         614656    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 1, 1, 256)         0         
_________________________________________________________________
conv_3 (Conv2D)              (None, 1, 1, 384)         885120    
__________

In [17]:
model.fit(x = X_train, y = Y_train, batch_size=batch_size, epochs=epochs,
              validation_data=[X_test,Y_test],shuffle = True,
              callbacks=[tensorboard])
# metrics = model.evaluate(X_test, Y_test)

Train on 404 samples, validate on 793 samples
Epoch 1/100
404/404 [==============================] - 4s 11ms/step - loss: 1.3847 - acc: 0.2723 - val_loss: 1.3761 - val_acc: 0.5826
Epoch 2/100
404/404 [==============================] - 5s 12ms/step - loss: 1.3729 - acc: 0.4059 - val_loss: 1.3619 - val_acc: 0.7150
Epoch 3/100
404/404 [==============================] - 5s 11ms/step - loss: 1.3523 - acc: 0.6238 - val_loss: 1.3400 - val_acc: 0.7238
Epoch 4/100
404/404 [==============================] - 4s 10ms/step - loss: 1.3304 - acc: 0.7228 - val_loss: 1.3121 - val_acc: 0.7137
Epoch 5/100
404/404 [==============================] - 6s 14ms/step - loss: 1.2970 - acc: 0.7574 - val_loss: 1.2663 - val_acc: 0.7037
Epoch 6/100
404/404 [==============================] - 5s 13ms/step - loss: 1.2432 - acc: 0.7797 - val_loss: 1.1975 - val_acc: 0.6961
Epoch 7/100
404/404 [==============================] - 5s 12ms/step - loss: 1.1743 - acc: 0.7995 - val_loss: 1.1095 - val_acc: 0.6873
Epoch 8/100
404/